<a href="https://colab.research.google.com/github/ddelizia/covid-19/blob/master/covid_19_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid Evolution Comunidad Valenciana

Data from: https://github.com/datadista/datasets/tree/master/COVID%2019

In [0]:
%pip install plotly

In [0]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [0]:
def normalize_data(dataset_url):
    df = pd.read_csv(dataset_url)
    df_ca = df.set_index('CCAA').drop(['cod_ine'], axis='columns').transpose()
    df_ca.index = pd.to_datetime(df_ca.index, format='%d/%m/%Y')
    return df_ca.reindex(pd.date_range(df_ca.index[0], df_ca.index[-1]))

df_cases = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv')
df_uci = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_uci.csv')
df_deaths = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv')
df_recovered = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_altas.csv')

In [115]:
ca = 'C. Valenciana'
data = pd.concat([df_cases[ca], df_deaths[ca], df_uci[ca], df_recovered[ca]], axis='columns').fillna(method='ffill').fillna(0)
data.columns = ["all", "deaths", "uci", "recovered"]
data['remaining'] = data["all"] - data["deaths"] - data["uci"] - data["recovered"]
data

,all,deaths,uci,recovered,remaining
2020-02-27,2,0.0,0.0,0.0,2.0
2020-02-28,8,0.0,0.0,0.0,8.0
2020-02-29,10,0.0,0.0,0.0,10.0
2020-03-01,15,0.0,0.0,0.0,15.0
2020-03-02,15,0.0,0.0,0.0,15.0
2020-03-03,15,0.0,0.0,0.0,15.0
2020-03-04,19,1.0,0.0,0.0,18.0
2020-03-05,19,1.0,0.0,0.0,18.0
2020-03-06,30,1.0,1.0,0.0,28.0
2020-03-07,30,1.0,1.0,0.0,28.0


In [121]:
fig = make_subplots(rows=3, cols=3, 
                    subplot_titles=(
    "Cases", "Hist cases", "",
    "Daily Delta", "Recovered Delta", "Deaths Delta",
    "Daily Delta %", "Recovered Delta %",  "Deaths Delta %"))

fig.add_trace(go.Scatter(y=data['all'], name="All cases"), row=1, col=1)

fig.add_trace(go.Bar(y=data['remaining'], name="Rest of the case cases"), row=1, col=2)
fig.add_trace(go.Bar(y=data['uci'], name="Ucis"), row=1, col=2)
fig.add_trace(go.Bar(y=data['deaths'], name="Deaths"), row=1, col=2)
fig.add_trace(go.Bar(y=data['recovered'], name="Recovered"), row=1, col=2)

fig.add_trace(go.Bar(y=data['all'].diff(), name="All case delta"), row=2, col=1)

fig.add_trace(go.Bar(y=100 * data['all'].diff().pct_change(), name="All cases delta %"), row=3, col=1)

fig.add_trace(go.Bar(y=data['recovered'].diff(), name="Recovered delta"), row=2, col=3)

fig.add_trace(go.Bar(y=100 * data['recovered'].diff().pct_change(), name="Recovered delta %"), row=3, col=3)

fig.add_trace(go.Bar(y=data['deaths'].diff(), name="Deaths delta"), row=2, col=2)

fig.add_trace(go.Bar(y=100 * data['deaths'].diff().pct_change(), name="Deaths delta %"), row=3, col=2)


fig.update_layout(height=1000,  title_text=f"Data {ca}")
fig.show()

In [118]:
############# PIE

labels = ['All cases','Ucis','Deaths', 'Recovered']
values = [data['remaining'][-1], data['uci'][-1], data['deaths'][-1], data['recovered'][-1]]

fig_pie = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig_pie.show()

In [147]:
x = np.linspace(0, data.shape[0] - 1, data.shape[0])
fig_log = go.Figure()

fig_log.add_trace(go.Scatter(y=data['all'], name="All cases on log scale"))
fig_log.add_trace(go.Scatter(y=np.power(2, x), name="Doubling daily", line=dict(width=1, dash='dash')))
fig_log.add_trace(go.Scatter(y=np.power(2, x/2), name="Doubling every 2 days", line=dict(width=1, dash='dash')))
fig_log.add_trace(go.Scatter(y=np.power(2, x/3), name="Doubling every 3 days", line=dict(width=1, dash='dash')))
fig_log.add_trace(go.Scatter(y=np.power(2, x/4), name="Doubling every 5 days", line=dict(width=1, dash='dash')))

fig_log.update_layout(yaxis_type="log", title_text=f"Log scale cases {ca}")
fig_log.show()